In [4]:
# import packages
import pandas as pd

# pip install mtgsdk
from mtgsdk import Card

In [25]:
# Import the previously scraped data, and create a master card list for adding additional data to and referencing later

# Maindeck import
maindeck_filepath = "data/maindeck_cards.csv"
maindeck_df = pd.read_csv(maindeck_filepath)

# Sideboard import
sideboard_filepath = "data/sideboard_cards.csv"
sideboard_df = pd.read_csv(sideboard_filepath)

# Concatenate only the 'card name' column from maindeck_df and sideboard_df
all_cards_df = pd.concat([maindeck_df[['card name']], sideboard_df[['card name']]], ignore_index=True)

# Keep only unique rows based on 'card name'
all_cards_df = all_cards_df.drop_duplicates(ignore_index=True)

# Add a unique id for each card
all_cards_df['card id'] = range(1, len(all_cards_df) + 1)

# Rearrange dataframe
all_cards_df = all_cards_df[['card id', 'card name']]

# Export to CSV
all_cards_df.to_csv("data/all_cards.csv", index=False)

all_cards_df


,card id,card name
0,1,Birds of Paradise
1,2,Brainstorm
2,3,Brazen Borrower
3,4,Collector Ouphe
4,5,Crashing Footfalls
...,...,...
843,844,Radiant Flames
844,845,Remorseful Cleric
845,846,Rending Volley
846,847,"Saheeli, Sublime Artificer"


In [31]:
# testing api

# cardname = all_cards_df['card name'][0]
cardname = "Snapcaster Mage"

try:
    cards = Card.where(name = cardname).all()
    exact_match = next((card for card in cards if card.name == cardname), cards[0])
    print(exact_match.name)
    print(exact_match.colors)
    print(int(exact_match.cmc))
    print(exact_match.type)
except IndexError:
    print(f"card not found - index error")
except Exception as e:
    print(f"An error occurred for card '{cardname}': {e}")

Snapcaster Mage
['U']
2
Creature — Human Wizard


In [29]:
# WARNING ------------------------
# Executing this cell will take 10+ minutes to fully collect the data.
# To see the final output, see the subsequent csv in data/card_info.csv
# Note that cards missing from the API are from the newest set

card_info = []
for index, row in all_cards_df.iterrows():
    cardname = row['card name']
    card_id = row['card id']
    try:
        cards = Card.where(name = cardname).all()
        exact_match = next((card for card in cards if card.name == cardname), cards[0])
        card_info.append({'card id': card_id, 'card name': cardname, 'colors': exact_match.colors, 'cmc': int(exact_match.cmc), 'type': exact_match.type})
        print(f"{row['card name']} information found. Next card.")
    except:
        card_info.append({'card id': card_id,'card name': cardname, 'colors': 'card not found', 'cmc': 'card not found', 'type': 'card not found'})
        print("------------")
        print(f"!!! {cardname} not found!")
        print("------------")


Birds of Paradise information found. Next card.
Brainstorm information found. Next card.
Brazen Borrower information found. Next card.
Collector Ouphe information found. Next card.
Crashing Footfalls information found. Next card.
Daze information found. Next card.
Delver of Secrets information found. Next card.
Dragon's Rage Channeler information found. Next card.
Drown in the Loch information found. Next card.
Elvish Spirit Guide information found. Next card.
Ezuri, Claw of Progress information found. Next card.
Fatal Push information found. Next card.
Fire // Ice information found. Next card.
Flooded Strand information found. Next card.
Force of Negation information found. Next card.
Force of Will information found. Next card.
Forest information found. Next card.
Grief information found. Next card.
Griselbrand information found. Next card.
Ice-Fang Coatl information found. Next card.
Island information found. Next card.
Lightning Bolt information found. Next card.
Lórien Revealed inf

In [33]:
# Show the additional data in a dataframe
card_info_df = pd.DataFrame(card_info)

# Store as a csv
card_info_df.to_csv("data/card_info.csv", index=False)

card_info_df

,card id,card name,colors,cmc,type
0,1,Birds of Paradise,[G],1,Creature — Bird
1,2,Brainstorm,[U],1,Instant
2,3,Brazen Borrower,[U],3,Creature — Faerie Rogue
3,4,Collector Ouphe,[G],2,Creature — Ouphe
4,5,Crashing Footfalls,[G],0,Sorcery
...,...,...,...,...,...
843,844,Radiant Flames,[R],3,Sorcery
844,845,Remorseful Cleric,[W],2,Creature — Spirit Cleric
845,846,Rending Volley,[R],1,Instant
846,847,"Saheeli, Sublime Artificer","[R, U]",3,Legendary Planeswalker — Saheeli


Next to-dos: 
1) drop card not found rows (or concat a manual spreadsheet)
2) split type and subtype 
3) SQL time!